In [1]:
!pip install fastai==0.7.0
!pip install torchtext==0.2.3

    100% |################################| 122kB 3.7MB/s 
    100% |################################| 496.4MB 101kB/s 
    100% |################################| 215kB 26.1MB/s 
    100% |################################| 3.6MB 12.3MB/s 
    100% |################################| 25.0MB 2.0MB/s 
    100% |################################| 71kB 24.0MB/s 
    100% |################################| 1.5MB 21.1MB/s 
    100% |################################| 11.6MB 4.7MB/s 
    100% |################################| 235kB 37.7MB/s 
    100% |################################| 61kB 26.5MB/s 
    100% |################################| 7.4MB 10.1MB/s 
    100% |################################| 81kB 33.7MB/s 
  Running setup.py bdist_wheel for feather-format ... done
  Stored in directory: /content/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
  Running setup.py bdist_wheel for bcolz ... done
  Stored in directory: /content/.cache/pip/wheels/9f/78/26/fb8c0

In [2]:
# target_url='https://s3.amazonaws.com/text-datasets/nietzsche.txt'

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [4]:
PATH='/content/clouderizer/nietzsche/data'

In [5]:
text = open('/content/clouderizer/nietzsche/data/nietzsche.txt', encoding="utf-8").read()
print('corpus length:', len(text))

corpus length: 600893


In [6]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


In [8]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [9]:
char_indices={c:i for i,c in enumerate(chars)}
indices_char={i:c for i,c in enumerate(chars)}

In [10]:
idx = [char_indices[c] for c in text]

idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [11]:
''.join(indices_char[i] for i in idx[:80])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor su'

In [12]:
cs=3
c1_dat=[idx[i] for i in range(0,len(idx)-cs)]

In [13]:
c1_dat[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [14]:
c1_dat=[idx[i] for i in range(0,len(idx)-cs,cs)]
c1_dat[:10]

[40, 30, 29, 1, 40, 43, 31, 61, 2, 74]

In [15]:
c2_dat=[idx[i+1] for i in range(0,len(idx)-cs,cs)]
c2_dat[:10]

[42, 25, 1, 43, 40, 33, 2, 54, 44, 73]

In [16]:
c3_dat=[idx[i+2] for i in range(0,len(idx)-cs,cs)]
c3_dat[:10]

[29, 27, 1, 45, 39, 38, 73, 73, 71, 61]

In [17]:
c4_dat=[idx[i+3] for i in range(0,len(idx)-cs,cs)]
c4_dat[:10]

[30, 29, 1, 40, 43, 31, 61, 2, 74, 2]

In [18]:
len(c1_dat)

200297

In [19]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

In [20]:
type(x1)

numpy.ndarray

In [21]:
y = np.stack(c4_dat)

In [22]:
# (x1[:4],x2[:4],x3[:4])

In [23]:
# y[:3]

In [24]:
y[:10]

array([30, 29,  1, 40, 43, 31, 61,  2, 74,  2])

In [25]:
y.shape, x1.shape

((200297,), (200297,))

In [26]:
n_hidden = 256
n_fac = 42

In [27]:
class Char3Model(nn.Module):
    def __init__(self,vocab_size,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in=nn.Linear(n_fac,n_hidden)
        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden=nn.Linear(n_hidden,n_hidden)
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out=nn.Linear(n_hidden,vocab_size)
        
    def forward(self,c1,c2,c3):
#         Calculationg the input activations
        in1=F.relu(self.l_in(self.e(c1)))
        in2=F.relu(self.l_in(self.e(c2)))
        in3=F.relu(self.l_in(self.e(c3)))
#         Calculating the hidden layer activations
        h=V(torch.zeros(in1.size()).cuda())
        h=F.tanh(self.l_hidden(h+in1))
        h=F.tanh(self.l_hidden(h+in2))
        h=F.tanh(self.l_hidden(h+in3))
                 
        return F.log_softmax(self.l_out(h))
        
                   

In [28]:
md=ColumnarModelData.from_arrays('.',[-1],np.stack([x1,x2,x3],axis=1),y,bs=512)
m=Char3Model(vocab_size, n_fac).cuda()

In [29]:
# vocab_size

In [30]:
# type(md)

In [31]:
it=iter(md.trn_dl)
*xs,yt=next(it)
t=m(*V(xs))

In [32]:
# xs,
len(xs)

3

In [33]:
xs[1].shape # batch size

torch.Size([512])

In [34]:
t  # spiiting out the probability of each possible vocab items

Variable containing:
-4.7143 -4.5457 -4.4375  ...  -4.4189 -4.3741 -4.4813
-4.7143 -4.5457 -4.4375  ...  -4.4189 -4.3741 -4.4813
-4.5835 -4.4796 -4.6344  ...  -4.2286 -4.1606 -4.4158
          ...             ⋱             ...          
-4.8649 -4.3647 -4.4089  ...  -4.1995 -4.3346 -4.3007
-4.6453 -4.5670 -4.3183  ...  -4.2115 -4.2231 -4.2507
-4.6931 -4.5005 -4.1621  ...  -4.1244 -4.4478 -4.4982
[torch.cuda.FloatTensor of size 512x85 (GPU 0)]

In [35]:
opt=optim.Adam(m.parameters(),1e-2)

In [36]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.129975   0.78639   



[array([0.78639])]

In [37]:
set_lrs(opt, 0.001)
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.86579    0.652874  



[array([0.65287])]

In [38]:
def get_next(inp):
    idxs=T(np.array([char_indices[i] for i in inp]))
    p=m(*VV(idxs))
    i=np.argmax(to_np(p))
    return chars[i]

In [39]:
get_next('y. ')

'T'

In [40]:
get_next('ppl')

'e'

# Our first RNN!

In [41]:
cs=8

In [42]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

In [43]:
c_in_dat[:8]

[[40, 42, 29, 30, 25, 27, 29, 1],
 [42, 29, 30, 25, 27, 29, 1, 1],
 [29, 30, 25, 27, 29, 1, 1, 1],
 [30, 25, 27, 29, 1, 1, 1, 43],
 [25, 27, 29, 1, 1, 1, 43, 45],
 [27, 29, 1, 1, 1, 43, 45, 40],
 [29, 1, 1, 1, 43, 45, 40, 40],
 [1, 1, 1, 43, 45, 40, 40, 39]]

In [44]:
# for j in range(len(idx)-cs):
#     for i in range(cs):
#         idx[i+j]

In [45]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]
c_out_dat[:5]

[1, 1, 43, 45, 40]

In [46]:
xs = np.stack(c_in_dat, axis=0)
xs.shape

(600885, 8)

In [47]:
y = np.stack(c_out_dat)
y.shape

(600885,)

In [48]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

In [49]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

In [50]:
a=[idx[j] for j in range(len(idx)-cs)]
''.join(str(a[:100]))

'[40, 42, 29, 30, 25, 27, 29, 1, 1, 1, 43, 45, 40, 40, 39, 43, 33, 38, 31, 2, 73, 61, 54, 73, 2, 44, 71, 74, 73, 61, 2, 62, 72, 2, 54, 2, 76, 68, 66, 54, 67, 9, 9, 76, 61, 54, 73, 2, 73, 61, 58, 67, 24, 2, 33, 72, 2, 73, 61, 58, 71, 58, 2, 67, 68, 73, 2, 60, 71, 68, 74, 67, 57, 1, 59, 68, 71, 2, 72, 74, 72, 69, 58, 56, 73, 62, 67, 60, 2, 73, 61, 54, 73, 2, 54, 65, 65, 2, 69, 61]'

# Create and train model

In [51]:
val_idx= get_cv_idxs(len(idx)-cs-1)

In [52]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [53]:
# class CharLoopModel(nn.Module):
#     def __init__(self,vocab_size,n_fac):
#         super().__init__()
#         self.e=nn.Embedding(vocab_size,n_fac)
        
#         self.l_in=nn.Linear(n_fac,n_hidden)
        
#         self.l_hidden=nn.Linear(n_hidden,n_hidden)
        
#         self.l_out=nn.Linear(n_hidden,vocab_size)
        
#     def forward(self,*cs):
#         bs=cs[0].size(0)
#         h=V(torch.zeros(bs,n_hidden).cuda())
#         for c in cs:
#             inp=F.relu(self.l_in(self.e))
#             h=F.tanh(self.l_hidden(h+inp))
#         return F.log_softmax(self.l_out(h),dim=1)

In [54]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
#         print("batch_size",bs)
        h = V(torch.zeros(bs, n_hidden).cuda())
#         print("h's dimension",h.size())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
#             print("inputs dimension",inp.size())
            h = F.tanh(self.l_hidden(h+inp))
#         print("output dim",self.l_out(h).size())
        return F.log_softmax(self.l_out(h), dim=-1)

In [55]:
m=CharLoopModel(vocab_size,n_fac).cuda()
opt=optim.Adam(m.parameters(),1e-2)

In [56]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.066089   2.037482  



[array([2.03748])]

In [57]:
set_lrs(opt,0.001)
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.78063    1.779354  



[array([1.77935])]

In [62]:
get_next('am plact')

' '

In [ ]:
 a woman--

# Concat Model

In [81]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [82]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [83]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [ ]:
fit(m, md, 1, opt, F.nll_loss)

 14%|█▍        | 131/939 [00:03<00:23, 34.01it/s, loss=2.6]

In [85]:
set_lrs(opt, 1e-4)
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.693773   1.703484  


[array([1.70348])]

In [86]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [87]:
get_next('for thos')

'e'

In [88]:
get_next('part of ')

't'

# RNN with pytorch

In [35]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [36]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [37]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [38]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([3, 512, 42])

In [39]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([3, 512, 256]), torch.Size([1, 512, 256]))

In [40]:
t = m(*V(xs)); t.size()

torch.Size([512, 85])

In [41]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.171821   1.245228  
    1      1.963483   0.849873  
    2      1.863857   0.390674  
    3      1.80519    0.39345   



[array([0.39345])]

In [42]:
set_lrs(opt, 1e-4)
fit(m, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.740068   0.365911  
    1      1.736439   0.360831  



[array([0.36083])]

In [43]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [44]:
get_next('for thos')

'e'

In [62]:
def get_next_n(inp, n):
    res = inp
#     
    for i in range(n):
        c = get_next(inp)
#         print(c)
        res += c
        inp = inp[1:]+c
#         print(inp)
    return res

In [61]:
get_next_n('for thos', 50)

'for thoseeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'

In [64]:
# inp='for thos'
# inp[1:]

# Multi-output model

In [95]:
cs=8

In [96]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

In [100]:
c_norm= [idx[j] for j in range(0, len(idx)-cs-1, cs)]

In [101]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [102]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [103]:
ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [104]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [105]:
ys[:cs,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

In [108]:
''.join(str(c_norm[:100]))

'[40, 1, 33, 2, 72, 67, 73, 2, 68, 57, 72, 2, 65, 68, 62, 71, 78, 58, 54, 61, 65, 67, 57, 9, 58, 65, 74, 67, 78, 74, 73, 2, 75, 65, 73, 57, 73, 8, 58, 62, 57, 65, 57, 62, 2, 27, 78, 72, 54, 61, 73, 67, 73, 73, 64, 57, 54, 61, 57, 71, 58, 62, 62, 72, 3, 58, 72, 2, 67, 54, 72, 8, 65, 2, 2, 74, 2, 61, 72, 2, 72, 73, 2, 65, 58, 68, 57, 68, 54, 68, 67, 62, 8, 58, 67, 75, 65, 67, 58, 62]'

# Create and train model

In [78]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [79]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [167]:
cs

8

In [158]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
#         self.e.shape
        self.rnn = nn.RNN(n_fac, n_hidden)
#         self.rnn.size()
        self.l_out = nn.Linear(n_hidden, vocab_size)
#         self.l_out.size()
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
#         print("Input size",inp.size())
        outp,h = self.rnn(inp, h)
#         print("Output size",outp.size())
        return F.log_softmax(self.l_out(outp))

In [159]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [160]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [161]:
# xst.size()

In [162]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [163]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.117653   0.949824  
    1      0.831512   0.761814  
    2      0.71066    0.679663  
    3      0.644108   0.630064  



[array([0.63006])]

In [151]:
set_lrs(opt, 1e-4)

In [152]:
fit(m, md, 1, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.994583   1.99814   



[array([1.99814])]

In [170]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    print(i)
    return chars

In [171]:
# def get_next_n(inp, n):
#     res = inp
# #     
#     for i in range(n):
#         c = get_next(inp)
# #         print(c)
#         res += c
#         inp = inp[1:]+c
# #         print(inp)
#     return res

In [172]:
get_next('for tho')

261


['\x00',
 '\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'Æ',
 'ä',
 'æ',
 'é',
 'ë']

# Identity init!

In [153]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [154]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [155]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.366349   2.208724  
    1      2.090662   2.025725  
    2      1.98293    1.956422  
    3      1.921595   1.922238  



[array([1.92224])]

In [156]:
set_lrs(opt, 1e-3)

In [157]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.82748    1.847171  
    1      1.815224   1.838688  
    2      1.808607   1.834527  
    3      1.801631   1.828492  



[array([1.82849])]